# **SDG Prediction Validation**

## **Dependencies**

In [3]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## **Load Data**

### Load GIZ Project Data As Ground Truth

GIZ Running Projects can be used as ground truth for SDG.

In [4]:
running_giz_df = pd.read_excel("../../src/ground_truths/giz_running_projects.xlsx")

running_giz_df.head(1)

,*Projektnummer,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,3. CRS-Bezeichnung,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt
0,2008.9076.4,Unterstützung bei der Einführung eines Kühlsch...,Refrigerator recycling system,41010.0,Umweltpolitik und -verwaltung,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13;15;,Unterstützung beim Aufbau eines beispielhaften...,Supporting the introduction of an exemplary ta...,"Umweltpolitik, Schutz und nachhaltige Nutzung ..."


### Load Merged DF

In [5]:
df = pd.read_csv("../../src/merged_orgas.csv")
df.head(1)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

### Extract Project Numbers out of IATI ID

In [ ]:
bmz_df = df[df.iati_orga_id == "DE-1"]

bmz_df["pn"] = "NaN"

for index, row in bmz_df.iterrows():
    try:
        pn_raw = row['iati_id'].split("-")[2]
        pn = f"{pn_raw[:4]}.{pn_raw[4:8]}.{pn_raw[8:]}"
        bmz_df.loc[index, "pn"] = pn
    except:
        pass

bmz_df.head(2)

C:\Users\UserNA6153\AppData\Local\Temp\ipykernel_9272\514410831.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bmz_df["pn"] = "NaN"


,iati_id,iati_orga_id,orga_abbreviation,orga_full_name,title_en,title_other,title_main,organization,country_code,country,...,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs,sgd_pred_code,sgd_pred_str,pn
0,DE-1-198670044,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,NaN,Studien- und Fachkräftefonds I; Fonds d&apos;E...,Studien- und Fachkräftefonds I; Fonds d&apos;E...,Bundesministerium für wirtschaftliche Zusammen...,['CD'],CD;,...,2024-01-31T00:00:00Z,2024-01-31T00:00:00Z,43010;,Multisector aid;,430;,Other Multisector;,['https://www.kfw-entwicklungsbank.de/Internat...,9,"8 Goal 9. Build resilient infrastructure, p...",1986.7004.4
1,DE-1-199370313,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,Studies and Experts Fund V,Studien- und Fachkräftefonds V,Studies and Experts Fund V,Bundesministerium für wirtschaftliche Zusammen...,['CN'],CN;,...,2024-01-31T00:00:00Z,2024-01-31T00:00:00Z,43010;,Multisector aid;,430;,Other Multisector;,['https://www.kfw-entwicklungsbank.de/Internat...,14,13 Goal 14. Conserve and sustainably use th...,1993.7031.3


### Extract running GIZ projects just with 1 sdg to achieve better compareability 

In [ ]:
running_giz_df = running_giz_df[running_giz_df['SDG Hauptziele'].str.len() == 3]

print(f"{len(running_giz_df)} with 1 SDG")
running_giz_df.head(1)

521 with 1 SDG


,*Projektnummer,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,3. CRS-Bezeichnung,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt
1,2010.9226.1,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU ...,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU b...,99810.0,Nicht spezifizierte Sektoren (fortgeführt als ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17;,EU Twinning,EU Twinning,Gestaltungsspielraum


### Match all running giz projects with IATI BMZ data 

In [ ]:
# rename project number column in running_giz_df
running_giz_df.rename(columns={'*Projektnummer': 'pn'}, inplace=True)
running_giz_df.head(1)

,pn,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,3. CRS-Bezeichnung,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt
1,2010.9226.1,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU ...,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU b...,99810.0,Nicht spezifizierte Sektoren (fortgeführt als ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17;,EU Twinning,EU Twinning,Gestaltungsspielraum


In [ ]:
# match with project number
merged_df = pd.merge(running_giz_df, bmz_df[["iati_id", "pn", "title_en", "description_main", "sgd_pred_code"]], on="pn", how="inner")
merged_df.dropna(subset=["sgd_pred_code"], inplace=True)
print(f"{len(merged_df)}")
merged_df.head(2)

420


,pn,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,...,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt,iati_id,title_en,description_main,sgd_pred_code
0,2014.0968.9,Globalvorhaben Ernährungssicherung und Resilie...,"Global programme Food and Nutrition Security, ...",43072.0,Programme zur Ernährungssicherung auf Haushalt...,0.7,43071.0,Politiken/Verwaltungsführung zur Ernährungssic...,0.3,NaN,...,NaN,NaN,02;,Die Ernährungssituation und die Resilienz gege...,The food situation and the resilience to hunge...,"Sicherung der Ernährung, Landwirtschaft",DE-1-201409689,"Global programme Food and Nutrition Security, ...",The food situation and the resilience to hunge...,7
1,2014.2456.3,Beschäftigung für nachhaltige Wirtschaftentwic...,Employment and Skills for Development in Africa,25010.0,Geschäftspolitik und -verwaltung,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,08;,Die Beschäftigungs- und wirtschaftliche Situat...,The employment- and economic situation of job ...,Nachhaltige Wirtschaftsentwicklung,DE-1-201424563,Employment and Skills for Development in Africa,The employment- and economic situation of job ...,13


In [ ]:
# just use most important columns
analy_df = merged_df[["pn", "Projektbezeichnung, deutsch", "title_en", "SDG Hauptziele", "sgd_pred_code"]]
analy_df.head(400)

,pn,"Projektbezeichnung, deutsch",title_en,SDG Hauptziele,sgd_pred_code
0,2014.0968.9,Globalvorhaben Ernährungssicherung und Resilie...,"Global programme Food and Nutrition Security, ...",02;,7
1,2014.2456.3,Beschäftigung für nachhaltige Wirtschaftentwic...,Employment and Skills for Development in Africa,08;,13
2,2014.4064.3,Verbesserung des Lernumfelds an öffentlichen S...,Improved Learning Environments at Public Schoo...,04;,9
3,2014.4065.0,Förderung partizipativen Ressourcenmanagements...,Supporting participatory Resource Management t...,06;,11
4,2014.4115.3,Initiative Kommunalentwicklung,Citizen-oriented urban development initiative,11;,2
...,...,...,...,...,...
395,2023.2165.1,GV Zentren für Migration und Entwicklung,Centres for Migration and Development,10;,13
396,2023.2166.9,Sektorvorhaben Globale Gesundheit,Sector Initiative Global Health,03;,8
397,2023.2169.3,Breitenwirksames Wirtschaftswachstum und Besch...,Improved conditions for economic diversification,08;,13
398,2023.2172.7,Globale Wassersicherheit für resiliente Entwic...,Global Water Security for Resilient Development,06;,11


In [ ]:
# change dtype that both cols can be compared
analy_df["SDG Hauptziele"] = analy_df["SDG Hauptziele"].str[:2].astype(int).astype(str)
analy_df["sgd_pred_code"] = analy_df["sgd_pred_code"].astype(str)
analy_df.head()

C:\Users\UserNA6153\AppData\Local\Temp\ipykernel_9272\3533025184.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analy_df["SDG Hauptziele"] = analy_df["SDG Hauptziele"].str[:2].astype(float)


,pn,"Projektbezeichnung, deutsch",title_en,SDG Hauptziele,sgd_pred_code
0,2014.0968.9,Globalvorhaben Ernährungssicherung und Resilie...,"Global programme Food and Nutrition Security, ...",2.0,7
1,2014.2456.3,Beschäftigung für nachhaltige Wirtschaftentwic...,Employment and Skills for Development in Africa,8.0,13
2,2014.4064.3,Verbesserung des Lernumfelds an öffentlichen S...,Improved Learning Environments at Public Schoo...,4.0,9
3,2014.4065.0,Förderung partizipativen Ressourcenmanagements...,Supporting participatory Resource Management t...,6.0,11
4,2014.4115.3,Initiative Kommunalentwicklung,Citizen-oriented urban development initiative,11.0,2


## **Analysis**

### Compare ground truth with predicted

In [ ]:
sdg_labels = [str(i) for i in range(1, 17)]

cm = confusion_matrix(y_true = analy_df['SDG Hauptziele'], 
                    y_pred = analy_df['sgd_pred_code'], 
                    labels = sdg_labels)

NameError: name 'confusion_matrix' is not defined

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(cm, fmt='d', cmap='Blues', xticklabels=sdg_labels, yticklabels=sdg_labels, annot=True, annot_kws={"size": 8})

plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()